# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import matplotlib.pyplot as plt
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from api_keys import weather_api_key
from api_keys import geoapify_key
gmaps.configure(api_key= geoapify_key)

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/WeatherPy.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mar del plata,-38.0023,-57.5575,59.88,84,0,23.00,AR,1667619566
1,1,castro,-24.7911,-50.0119,44.91,92,1,5.21,BR,1667619644
2,2,punta arenas,-53.1500,-70.9167,53.71,71,75,12.66,CL,1667619645
3,3,broken hill,-31.9500,141.4333,72.23,28,100,4.45,AU,1667619645
4,4,flinders,-34.5833,150.8552,73.06,65,59,12.06,AU,1667619646


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
%%capture --no-display

# Configure the map plot
locations = city_data_df[['Lat', 'Lng']]
weights = city_data_df['Humidity']
central_coordinates = (0, 0)
fig = gmaps.figure(center=central_coordinates, zoom_level=1.5)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, max_intensity=100, point_radius=2, dissipating=False)
heatmap_layer.opacity = 0.5
fig.add_layer(heatmap_layer)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data=city_data_df

# A max temperature lower than 75 degrees but higher than 65.
filtered_city_data = filtered_city_data.loc[(filtered_city_data["Max Temp"] < 75) & (filtered_city_data["Max Temp"] > 65)]

# Wind speed less than 10 mph.
filtered_city_data = filtered_city_data.loc[filtered_city_data["Wind Speed"] < 10]

# Zero cloudiness.
filtered_city_data = filtered_city_data.loc[filtered_city_data["Cloudiness"] == 0]

# Drop any rows with null values
filtered_city_data = filtered_city_data.dropna()

# Display sample data
filtered_city_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,adrar,20.5022,-10.0711,68.86,16,0,8.28,MR,1667619664
63,63,busselton,-33.6500,115.3333,69.76,41,0,1.72,AU,1667619674
93,93,san matias,-16.3667,-58.4000,65.03,53,0,2.66,BO,1667619687
121,121,guerrero negro,27.9769,-114.0611,65.35,40,0,8.10,MX,1667619699
165,165,frontera,26.9333,-101.4500,71.33,39,0,3.00,MX,1667619717
208,208,vila velha,-20.3297,-40.2925,67.95,80,0,9.22,BR,1667619734
373,373,fort stewart,31.8722,-81.6100,69.75,68,0,4.61,US,1667619799
383,383,brufut,13.3814,-16.7518,73.49,94,0,3.44,GM,1667619804
401,401,pirgos,35.1167,25.7178,66.06,62,0,6.13,GR,1667619813
422,422,huntington,38.4193,-82.4452,70.70,66,0,9.22,US,1667619741


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
45,adrar,MR,20.5022,-10.0711,16,
63,busselton,AU,-33.6500,115.3333,41,
93,san matias,BO,-16.3667,-58.4000,53,
121,guerrero negro,MX,27.9769,-114.0611,40,
165,frontera,MX,26.9333,-101.4500,39,
208,vila velha,BR,-20.3297,-40.2925,80,
373,fort stewart,US,31.8722,-81.6100,68,
383,brufut,GM,13.3814,-16.7518,94,
401,pirgos,GR,35.1167,25.7178,62,
422,huntington,US,38.4193,-82.4452,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
params = {
    "radius": 10000,
    "type": "lodging",
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
    
   # Set base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotel = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    except (IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

Starting hotel search
adrar - nearest hotel: No hotel found
busselton - nearest hotel: Observatory Guest House
san matias - nearest hotel: HOTEL "SAN JOSE"
guerrero negro - nearest hotel: The Halfway Inn
frontera - nearest hotel: Fiesta Inn Monclova
vila velha - nearest hotel: Píer Vitória Hotel - Hotel em Vitória/ES
fort stewart - nearest hotel: Holiday Inn Express & Suites Hinesville East - Fort Stewart, an IHG Hotel
brufut - nearest hotel: Gambia Hotel Resort & Spa
pirgos - nearest hotel: Vangelis Villas
huntington - nearest hotel: Ramada Limited Huntington
cabo san lucas - nearest hotel: Hotel Tesoro Los Cabos
atar - nearest hotel: Oumou Elghoura
pokhara - nearest hotel: OYO 170 Hotel View Point
les cayes - nearest hotel: Villa Orphee
boa vista - nearest hotel: Aipana Plaza Hotel


In [25]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
45,adrar,MR,20.5022,-10.0711,16,No hotel found
63,busselton,AU,-33.6500,115.3333,41,Observatory Guest House
93,san matias,BO,-16.3667,-58.4000,53,"HOTEL ""SAN JOSE"""
121,guerrero negro,MX,27.9769,-114.0611,40,The Halfway Inn
165,frontera,MX,26.9333,-101.4500,39,Fiesta Inn Monclova
208,vila velha,BR,-20.3297,-40.2925,80,Píer Vitória Hotel - Hotel em Vitória/ES
373,fort stewart,US,31.8722,-81.6100,68,Holiday Inn Express & Suites Hinesville East -...
383,brufut,GM,13.3814,-16.7518,94,Gambia Hotel Resort & Spa
401,pirgos,GR,35.1167,25.7178,62,Vangelis Villas
422,huntington,US,38.4193,-82.4452,66,Ramada Limited Huntington


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """

Hotel Name
{Hotel Name}

City
{City}
Country
{Country}

"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))